In [41]:
import os
import json
import nibabel as nib
import numpy as np
from nibabel.processing import resample_from_to

In [14]:
# Load config file with path to raw, derived and output data
config_file = os.path.join(os.path.abspath(''), os.pardir, 'scripts', 'config.json')
with open(config_file) as json_data:
    config = json.load(json_data)

out_dir = config['output_dir']

In [8]:
fsl_dir = os.environ['FSLDIR']
atlas_file = os.path.join(fsl_dir, 'data', 'atlases', 'HarvardOxford', 'HarvardOxford-cort-maxprob-thr0-2mm.nii.gz')

In [11]:
atlas = nib.load(atlas_file)

In [44]:
res_dir = os.path.join(out_dir, 'SPM', 'level3')
gain_dir = os.path.join(res_dir, 'gain')
loss_dir = os.path.join(res_dir, 'loss')

gain_indif = nib.load(os.path.join(gain_dir, 'spmT_0001_thresh_.nii')).get_data()
gain_range = nib.load(os.path.join(gain_dir, 'spmT_0002_thresh_.nii')).get_data()

loss_indif_neg = nib.load(os.path.join(loss_dir, 'spmT_0001_thresh_.nii')).get_data()
loss_range_neg = nib.load(os.path.join(loss_dir, 'spmT_0002_thresh_.nii')).get_data()
loss_indif = nib.load(os.path.join(loss_dir, 'spmT_0003_thresh_.nii')).get_data()
loss_range = nib.load(os.path.join(loss_dir, 'spmT_0004_thresh_.nii')).get_data()
loss_range_vs_indif = nib.load(os.path.join(loss_dir, 'spmT_0005_thresh_.nii')).get_data()

In [36]:
print(atlas.header.get_data_shape())
print(gain_indif.header.get_data_shape())

# Reslice atlas on SPM's MNI default resolution using nearest neighbours
atlas_spm = resample_from_to(atlas, gain_indif, order=0)
print(atlas_spm.header.get_data_shape())

(91, 109, 91)
(97, 115, 81)
(97, 115, 81)


In [43]:
VMPFC = atlas_spm.get_data() == 25
VStriatum = np.logical_or(atlas_spm.get_data() == 11, atlas_spm.get_data() == 21)
Amygdala = np.logical_or(atlas_spm.get_data() == 10, atlas_spm.get_data() == 20)

In [55]:
# 1. Parametric effect of gain: Positive effect in ventromedial PFC - for the equal indifference group
print('1. Parametric effect of gain: Positive effect in ventromedial PFC - for the equal indifference group')
hyp1 = sum(sum(sum(np.logical_and((gain_indif>0), VMPFC))))
print(hyp1)

# 2. Parametric effect of gain: Positive effect in ventromedial PFC - for the equal range group
print('2. Parametric effect of gain: Positive effect in ventromedial PFC - for the equal range group')
hyp2 = sum(sum(sum(np.logical_and((gain_range>0), VMPFC))))
print(hyp2)

# 3. Parametric effect of gain: Positive effect in ventral striatum - for the equal indifference group
print('3. Parametric effect of gain: Positive effect in ventral striatum - for the equal indifference group')
hyp3 = sum(sum(sum(np.logical_and((gain_indif>0), VStriatum))))
print(hyp3)

# 4. Parametric effect of gain: Positive effect in ventral striatum - for the equal range group
print('4. Parametric effect of gain: Positive effect in ventral striatum - for the equal range group')
hyp4 = sum(sum(sum(np.logical_and((gain_range>0), VStriatum))))
print(hyp4)

# 5. Parametric effect of loss: Negative effect in VMPFC - for the equal indifference group
print('5. Parametric effect of loss: Negative effect in VMPFC - for the equal indifference group')
hyp5 = sum(sum(sum(np.logical_and((loss_indif_neg>0), VMPFC))))
print(hyp5)

# 6. Parametric effect of loss: Negative effect in VMPFC - for the equal range group
print('6. Parametric effect of loss: Negative effect in VMPFC - for the equal range group')
hyp6 = sum(sum(sum(np.logical_and((loss_range_neg>0), VMPFC))))
print(hyp6)

# 7. Parametric effect of loss: Positive effect in amygdala - for the equal indifference group
print('7. Parametric effect of loss: Positive effect in amygdala - for the equal indifference group')
hyp7 = sum(sum(sum(np.logical_and((loss_indif>0), Amygdala))))
print(hyp7)

# 8. Parametric effect of loss: Positive effect in amygdala - for the equal range group
print('8. Parametric effect of loss: Positive effect in amygdala - for the equal range group')
hyp8 = sum(sum(sum(np.logical_and((loss_range>0), Amygdala))))
print(hyp8)

# 9. Equal range vs. equal indifference: Greater response to losses in amygdala for equal range condition vs. equal indifference condition
print('9. Equal range vs. equal indifference: Greater response to losses in amygdala for equal range condition vs. equal indifference condition')
hyp9 = sum(sum(sum(np.logical_and((loss_range_vs_indif>0), Amygdala))))
print(hyp9)

1. Parametric effect of gain: Positive effect in ventromedial PFC - for the equal indifference group
1
2. Parametric effect of gain: Positive effect in ventromedial PFC - for the equal range group
0
3. Parametric effect of gain: Positive effect in ventral striatum - for the equal indifference group
0
4. Parametric effect of gain: Positive effect in ventral striatum - for the equal range group
560
5. Parametric effect of loss: Negative effect in VMPFC - for the equal indifference group
403
6. Parametric effect of loss: Negative effect in VMPFC - for the equal range group
0
7. Parametric effect of loss: Positive effect in amygdala - for the equal indifference group
0
8. Parametric effect of loss: Positive effect in amygdala - for the equal range group
0
9. Equal range vs. equal indifference: Greater response to losses in amygdala for equal range condition vs. equal indifference condition
0


/anaconda3/envs/narps/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in greater
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/envs/narps/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in greater
  
/anaconda3/envs/narps/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in greater
  del sys.path[0]
/anaconda3/envs/narps/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in greater
/anaconda3/envs/narps/lib/python3.6/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in greater
/anaconda3/envs/narps/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in greater
/anaconda3/envs/narps/lib/python3.6/site-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in grea

In [50]:
hyp1

1